In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch

/Users/tristan/miniconda3/envs/deep/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [97]:
class AutoEncoder(nn.Module):
    def __init__(self, cnn_kernel=1, cnn_stride=1, mp_kernel=6, mp_stride=1, lstm_hidden_dim=3):
        super().__init__()
        self.lstm_hidden_dim = lstm_hidden_dim

        self.cnn_one = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=cnn_kernel, stride=cnn_stride, padding=0)
        self.max_pool_one = nn.MaxPool1d(kernel_size=mp_kernel, stride=mp_stride, padding=0)
        
        self.bi_lstm = nn.LSTM(
            input_size=1,
            hidden_size=lstm_hidden_dim,
            num_layers=2,
            batch_first=True,
            dropout=0,
            bidirectional=True,
        )


        self.upsample = nn.Upsample(scale_factor=mp_kernel)

        # self.upsample = nn.Upsample(size=mp_kernel, mode='nearest')
        # self.up_layer = nn.Upsample(size=3)
        # self.deconv_layer = nn.ConvTranspose1d(
        #     in_channels=lstm_hidden_dim,
        #     out_channels=lstm_hidden_dim,
        #     kernel_size=10,
        #     stride=1,
        #     padding=mp_kernel // 2,
        # )
        # self.deconv = nn.ConvTranspose1d(in_channels=1, out_channels=1,)

    def forward(self, x):
        print(x.shape)
        x = x.permute(0, 2, 1)
        print(x.shape)
        print('-'*50)



        x = F.leaky_relu(self.cnn_one(x))
        print(x.shape)
        print('-'*50)
        x = self.max_pool_one(x)
        
        
        print(x.shape)
        x = x.permute(0, 2, 1)
        print(x.shape)
        print('-'*50)



        x, (_, _) = self.bi_lstm(x)

        print(x.shape)
        print('-'*50)


        x = torch.sum(
            x.view(
                x.shape[0], x.shape[1], 2, self.lstm_hidden_dim
            ),
            dim=2,
        )

        print(x.shape)
        print('-'*50)

        # print(x)
        # print('-'*50)

        # x = self.up_layer(x)

        # print(x)
        # print('-'*50)

        # x = self.deconv_layer(x)

        # print(x)



        # Forward and backward outputs summed
        # Can't be concatenated as clustering level expected 1D series
        # Paper doesn't specify
        # x = x[:, :, :self.lstm_hidden_dim] + x[:, :, self.lstm_hidden_dim:]
        # print(x)
        # print('-'*50)

        # print(x.shape)

        # x = self.upsample(x)
        # print(x.shape)
        # print(x)

        # return x





In [98]:
x = torch.tensor([
    [[1]] * 10,
    [[1]] * 10,
]).to(torch.float32)

In [99]:
x.shape

torch.Size([2, 10, 1])

In [100]:
network = AutoEncoder()

In [101]:
network(x)

torch.Size([2, 10, 1])
torch.Size([2, 1, 10])
--------------------------------------------------
torch.Size([2, 1, 10])
--------------------------------------------------
torch.Size([2, 1, 5])
torch.Size([2, 5, 1])
--------------------------------------------------
torch.Size([2, 5, 6])
--------------------------------------------------
torch.Size([2, 5, 3])
--------------------------------------------------


In [36]:
x = torch.tensor([[[1,1]], [[1,1]]]).to(torch.float32)

In [37]:
x

tensor([[[1., 1.]],

        [[1., 1.]]])

In [38]:
layer = nn.Upsample(scale_factor=2)

In [39]:
layer(x)

tensor([[[1., 1., 1., 1.]],

        [[1., 1., 1., 1.]]])

In [360]:
x = torch.tensor([
    [[1]] * 5,
    [[1]] * 5,
]).to(torch.float32)

In [361]:
x.shape

torch.Size([2, 5, 1])

In [362]:
l1 = nn.Conv1d(1, 1, 3, 1)

In [363]:
x = x.permute(0, 2, 1)
x = l1(x)

In [364]:
x.shape

torch.Size([2, 1, 3])

In [365]:
l1 = nn.MaxPool1d(2, 1)

In [366]:
x = l1(x)

In [367]:
x.shape

torch.Size([2, 1, 2])

In [368]:
x = x.permute(0,2,1)

In [369]:
x.shape

torch.Size([2, 2, 1])

In [370]:
l3 = nn.LSTM(
    input_size=1,
    hidden_size=2,
    num_layers=2,
    batch_first=True,
    dropout=0,
    bidirectional=True,
)

In [371]:
x, (_, _) = l3(x)

In [372]:
x.shape

torch.Size([2, 2, 4])

When bidirectional=True, output will contain a concatenation of the forward and reverse hidden states at each time step in the sequence.

In [373]:
x 

tensor([[[-0.2267, -0.1067,  0.1162,  0.1304],
         [-0.3676, -0.2266,  0.0896,  0.1472]],

        [[-0.2267, -0.1067,  0.1162,  0.1304],
         [-0.3676, -0.2266,  0.0896,  0.1472]]], grad_fn=<TransposeBackward0>)

In [374]:
# Merge forward and reverse by adding them
x = x[:, :, :2] + x[:, :, 2:]

In [375]:
x.shape

torch.Size([2, 2, 2])

In [376]:
l4  = nn.Upsample(scale_factor=2)

In [377]:
# Upsample time dimension, not features
x = x.permute(0,2,1)
x = l4(x)
x = x.permute(0,2,1)

In [378]:
x.shape

torch.Size([2, 4, 2])

In [387]:
l5 = nn.ConvTranspose1d(
    in_channels=2,
    out_channels=1,
    kernel_size=2,
    stride=1,
    padding=0,
)

In [388]:
y = x.permute(0,2,1)
y = l5(y)
y = y.permute(0,1,2)

In [389]:
y.shape

torch.Size([2, 1, 5])

In [390]:
y

tensor([[[-0.1899, -0.2700, -0.3802, -0.4597, -0.2733]],

        [[-0.1899, -0.2700, -0.3802, -0.4597, -0.2733]]],
       grad_fn=<PermuteBackward0>)

In [475]:
class Network(nn.Module):
    def __init__(self, cnn_kernel):
        super().__init__()

        self.cnn = nn.Conv1d(1, 1, 3, 1, 0)
        self.pool = nn.MaxPool1d(2, 1)

        self.lstm = nn.LSTM(
            input_size=1,
            hidden_size=2,
            num_layers=2,
            batch_first=True,
            dropout=0,
            bidirectional=True,
        )

        self.upsample = nn.Upsample(scale_factor=2)
        self.trans_cnn = nn.ConvTranspose1d(
            in_channels=2,
            out_channels=1,
            kernel_size=2,
            stride=1,
            padding=0,
        )

    def forward(self, x):
        x = x.permute(0,2,1)
        x = F.relu(self.cnn(x))
        x = self.pool(x)
        
        x = x.permute(0,2,1)
        x, (_, _) = self.lstm(x)
        x = x[:, :, :2] + x[:, :, 2:]

        x = x.permute(0,2,1)
        x = self.upsample(x)
        # x = x.permute(0,2,1)
        
        # x = x.permute(0,2,1)
        print(x.shape)
        x = self.trans_cnn(x)
        x = x.permute(0,2,1)
        return x

In [476]:
x = torch.tensor([
    [[1]] * 5,
    [[1]] * 5,
]).to(torch.float32)

In [477]:
x.shape

torch.Size([2, 5, 1])

In [478]:
y = Network(cnn_kernel=3)(x)

torch.Size([2, 2, 4])


In [479]:
y.shape

torch.Size([2, 5, 1])